In [1]:
#On importe les librairies nécessaires pour faire fonctionner le notebook 

import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import os
import plotly.io as pio

pio.renderers.default="vscode+pdf"
pio.templates["custom"] = pio.templates["plotly_white"]
pio.templates["custom"]["layout"]["font"] = {"size": 15}
pio.templates.default = "custom"

# Analyse des simulations

Deux simulations ont été effectuées, une avec les données GTFS datant du 17/02/2025 et l'autre avec l'ajout des nouvelles lignes au-dit GTFS.

Les résultats sont disponibles dans les dossiers baseline_output et gpe_output.

Les simulations ont été faites sur un échantillonage de 1% de la population de l'Île-de-France. On multiplie donc les résultats par un facteur 100 pour avoir les résultats à l'échelle.

In [2]:
# Load les données 

gtfs_folder = r"..\implementation_gtfs\GTFS_versions\GTFS_completed" # Il faudrait mettres les données GTFS dans un sous dossier ici.
routes = pd.read_csv(os.path.join(gtfs_folder, "routes.txt"))

baseline_folder = os.path.join("baseline_output","1_percent_june")
baseline_legs = pd.read_csv(os.path.join(baseline_folder, "eqasim_legs.csv"), sep = ";")
baseline_pt = pd.read_csv(os.path.join(baseline_folder, "eqasim_pt.csv"), sep = ";")
baseline_trips = pd.read_csv(os.path.join(baseline_folder, "eqasim_trips.csv"), sep = ";")

gpe_folder = os.path.join("gpe_output", "1_percent_june")
gpe_legs = pd.read_csv(os.path.join(gpe_folder, "eqasim_legs.csv"), sep = ";")
gpe_pt = pd.read_csv(os.path.join(gpe_folder, "eqasim_pt.csv"), sep = ";")
gpe_trips = pd.read_csv(os.path.join(gpe_folder, "eqasim_trips.csv"), sep = ";")


In [3]:
# On récupère tous les modes

modes = set(baseline_trips["mode"].unique()).union(set(gpe_trips["mode"].unique()))
pt_modes = set(baseline_pt["transit_mode"].unique()).union(set(gpe_pt["transit_mode"].unique()))

In [4]:
# Gestion des données
fact_baseline = 100
fact_gpe = 100

count_modes_baseline = {mode: len(baseline_trips[baseline_trips["mode"] == mode])*fact_baseline for mode in modes}
count_modes_gpe = {mode: len(gpe_trips[gpe_trips["mode"] == mode])*fact_gpe for mode in modes}

count_modes = pd.DataFrame({
    "baseline" : count_modes_baseline,
    "gpe" : count_modes_gpe
})

count_modes = count_modes.sort_values(by='baseline', ascending=True)

diff_counts = {mode: count_modes_gpe[mode] - count_modes_baseline[mode] for mode in modes}

diff_percents = {
    mode: (diff_counts[mode] / count_modes_baseline[mode] * 100) if count_modes_baseline[mode] != 0 else float('inf')
    for mode in modes
}

diff_counts = pd.DataFrame({
    "gpe - baseline" : dict(sorted(diff_counts.items(), key=lambda item: item[1]))
})

diff_percents= pd.DataFrame({
    "percentage_diff" : dict(sorted(diff_percents.items(), key=lambda item: item[1]))
})

count_pt_modes_baseline = {mode: len(baseline_pt[baseline_pt["transit_mode"] == mode])*fact_baseline for mode in pt_modes}
count_pt_modes_gpe = {mode: len(gpe_pt[gpe_pt["transit_mode"] == mode])*fact_gpe for mode in pt_modes}

count_pt_modes = pd.DataFrame({
    "baseline" : count_pt_modes_baseline,
    "gpe" : count_pt_modes_gpe
})

count_pt_modes = count_pt_modes.sort_values(by='baseline', ascending=True)

diff_counts_pt = {mode: count_pt_modes_gpe[mode] - count_pt_modes_baseline[mode] for mode in pt_modes}
diff_counts_pt = pd.DataFrame({
    "gpe - baseline" : dict(sorted(diff_counts_pt.items(), key=lambda item: item[1]))
})

In [5]:
fig = px.bar(count_modes.rename(
                index={
                    "bike" : "Vélo",
                    "car_passenger" : "Voiture (passager)",
                    "car" : "Voiture (conducteur)",
                    "pt" : "Transport en commun",
                    "walk" : "Marche"
                    },
                columns={
                    "baseline" : "Simulation de référence", 
                    "gpe" : "Avec GPE"
                    }
                ),
            barmode="group", 
            labels={
                 "index" : "Mode de transport",
                 "value" : "Nombre de trajets",
                 "variable" : "Simulation",
            },
            title="Nombre de trajets en fonction du mode de transport"
            )

fig.write_image("outputs/plots/comparaison/trajets_transport.png", width=1000)
fig.show()

Sur la figure ci-dessus, on voit qu'il y a très peu de changement au niveau des modes de transport pris entre la simulation de référence et la simulation incluant les lignes du GPE.

In [6]:
fig = px.bar(diff_counts.rename(
                index={
                    "bike" : "Vélo",
                    "car_passenger" : "Voiture (passager)",
                    "car" : "Voiture (conducteur)",
                    "pt" : "Transport en commun",
                    "walk" : "Marche"
                    },
                columns={
                    "gpe - baseline" : "Variation"
                }),
             labels={
                 "value" : "Variation du nombre de trajets",
                 "index" : "Mode de transport"
             },
             title="Variation du nombre de trajets effectués par mode de transport")

fig.write_image("outputs/plots/comparaison/variation_trajets_transport.png", width=1000)
fig.show()

In [7]:
fig = px.bar(diff_percents.rename(
                index={
                    "bike" : "Vélo",
                    "car_passenger" : "Voiture (passager)",
                    "car" : "Voiture (conducteur)",
                    "pt" : "Transport en commun",
                    "walk" : "Marche"
                    },
                columns={
                    "percentage_diff" : "Variation"
                }),
             labels={
                 "value" : "Variation du nombre de trajets",
                 "index" : "Mode de transport"
             },
            title="Variation en % du nombre de trajets effectués par mode de transport")

fig.write_image("outputs/plots/comparaison/variation_trajets_transport_percent.png", width=1000)
fig.show()

La figure ci-dessus présente de façon plus lisible la différence entre les deux simulations. Les lignes du GPE ont donc amené une augmentation du nombre de trajets en transport en commun tout en faisant diminuer les trajets en voitures.

In [8]:
fig = px.bar(count_pt_modes.rename(
                index={
                    "tram" : "Tramway",
                    "rail" : "Train (RER, TER, Transilien)",
                    "subway" : "Métro",
                    "bus" : "Bus"
                    },
                columns={
                    "baseline" : "Simulation de référence",
                    "gpe" : "Avec GPE"
                }),
            barmode="group",
            labels={
                "variable" : "Simulation",
                "value" : "Nombre de trajets",
                "index" : "Mode de transport en commun"
            },
            #title="Nombre de trajets effectués par mode de transport en commun"
            )
fig.write_image("outputs/plots/comparaison/trajets_transport_commun.png", width=1000)
fig.show()

Si on se concentre plus sur les transports en commun on voit que les lignes du GPE ont eu un impact significatif sur la prise des modes de transports en commun.

In [9]:
fig = px.bar(diff_counts_pt.rename(
                index={
                    "tram" : "Tramway",
                    "rail" : "Train (RER, TER, Transilien)",
                    "subway" : "Métro",
                    "bus" : "Bus"
                    },
                columns={
                    "gpe - baseline" : "Variation"
                }),
             labels={
                 "value" : "Variation du nombre de trajets",
                 "index" : "Mode de transport en commun"
             },
             #title="Variation du nombre de trajets effectués par mode de transport en commun"
             )
fig.write_image("outputs/plots/comparaison/variation_trajets_transport_commun.png", width=1000)
fig.show()

Ici on voit une augmentation nette du nombre de trajets en métro ainsi qu'une diminution du nombre de trajets dans les trois autres modes.

In [10]:
diff_percent = ((count_pt_modes["gpe"] - count_pt_modes["baseline"]) / count_pt_modes["baseline"]) * 100

# Renommer les index pour le graphique
diff_percent = diff_percent.rename(
    index={
        "bike": "Vélo",
        "car_passenger": "Voiture (passager)",
        "car": "Voiture (conducteur)",
        "pt": "Transport en commun",
        "walk": "Marche"
    }
)

# Création du graphique
fig = px.bar(diff_percent,
             labels={
                 "value": "Variation en pourcentage du nombre de trajets",
                 "index": "Mode de transport"
             },
             title="Variation en pourcentage du nombre de trajets effectués par mode de transport")

# Sauvegarde de l'image
#fig.write_image("outputs/plots/comparaison/variation_pourcentage_trajets_transport.png", width=1000)

# Affichage du graphique
fig.show()

Cette évolution correspond à une augmentation de 12% du nombre de trajets en métro et d'une diminution d'entre 4 et 5% pour les trois autres modes de transport. On observe donc un effet de désengorgement des lignes hors métro.

In [11]:
total_baseline = sum(count_modes_baseline.values())
total_gpe = sum(count_modes_gpe.values())
percentages_baseline = {mode: (count / total_baseline) * 100 for mode, count in count_modes_baseline.items()}
percentages_gpe = {mode: (count / total_gpe) * 100 for mode, count in count_modes_gpe.items()}

percentages_baseline_df = pd.DataFrame(list(percentages_baseline.items()), columns=['Mode', 'Percentage'])
percentages_gpe_df = pd.DataFrame(list(percentages_gpe.items()), columns=['Mode', 'Percentage'])

fig_baseline = px.pie(percentages_baseline_df, values='Percentage', names='Mode', title='Pourcentages des modes (Control)')
fig_gpe = px.pie(percentages_gpe_df, values='Percentage', names='Mode', title='Pourcentages des modes (GPE)')

fig = make_subplots(rows=1, cols=2, specs=[[{'type':'domain'}, {'type':'domain'}]],
                    subplot_titles=['Sans GPE', 'Avec GPE'])

fig.add_trace(go.Pie(labels=percentages_baseline_df['Mode'], values=percentages_baseline_df['Percentage']), 1, 1)
fig.add_trace(go.Pie(labels=percentages_gpe_df['Mode'], values=percentages_gpe_df['Percentage']), 1, 2)

fig.update_layout(title_text='Comparaison des pourcentages des modes (Sans vs Avec GPE)')

fig.show()

On voit ici que la répartition des trajets en fonction des modes de transport est très semblable avec et sans GPE. L'ajout des lignes du GPE seules n'a donc pas permis de modifier considérablement les choix des agents sur les modes de transport.

In [12]:
total_baseline = sum(count_pt_modes_baseline.values())
total_gpe = sum(count_pt_modes_gpe.values())
percentages_baseline = {mode: (count / total_baseline) * 100 for mode, count in count_pt_modes_baseline.items()}
percentages_gpe = {mode: (count / total_gpe) * 100 for mode, count in count_pt_modes_gpe.items()}

percentages_baseline_df = pd.DataFrame(list(percentages_baseline.items()), columns=['Mode', 'Percentage'])
percentages_gpe_df = pd.DataFrame(list(percentages_gpe.items()), columns=['Mode', 'Percentage'])

fig_baseline = px.pie(percentages_baseline_df, values='Percentage', names='Mode', title='Pourcentages des modes (Control)')
fig_gpe = px.pie(percentages_gpe_df, values='Percentage', names='Mode', title='Pourcentages des modes (GPE)')

fig = make_subplots(rows=1, cols=2, specs=[[{'type':'domain'}, {'type':'domain'}]],
                    subplot_titles=['Sans GPE', 'Avec GPE'])

fig.add_trace(go.Pie(labels=percentages_baseline_df['Mode'], values=percentages_baseline_df['Percentage']), 1, 1)
fig.add_trace(go.Pie(labels=percentages_gpe_df['Mode'], values=percentages_gpe_df['Percentage']), 1, 2)

fig.update_layout(title_text='Comparaison des pourcentages des modes de transport en commun (Sans vs Avec GPE)')

fig.show()

Ici on retrouve le même effet exprimé avant, où l'on remarque un changement significatif. Les agents ont tendance à plus choisir les lignes de métro avec l'ajout des lignes que dans la simulation de référence.

In [13]:
count_lines_baseline = {mode: {} for mode in pt_modes if mode != "bus"}
count_lines_gpe = {mode: {} for mode in pt_modes if mode != "bus"}

for mode in pt_modes:
    if mode != "bus":
        tmp_baseline = baseline_pt[baseline_pt.transit_mode == mode]
        tmp_gpe = gpe_pt[gpe_pt.transit_mode == mode]
        lines_baseline = tmp_baseline["transit_line_id"].unique()
        lines_gpe = tmp_gpe["transit_line_id"].unique()
        lines = set(lines_baseline).union(set(lines_gpe))

        for line in lines:
            name = routes[routes.route_id == line]
            if not name.empty:
                name = name["route_short_name"].values[0]
                if name not in count_lines_baseline[mode]:
                    count_lines_baseline[mode][name] = 0
                if name not in count_lines_gpe[mode]:
                    count_lines_gpe[mode][name] = 0
                count_lines_baseline[mode][name] += len(tmp_baseline[tmp_baseline.transit_line_id == line])*fact_baseline
                count_lines_gpe[mode][name] += len(tmp_gpe[tmp_gpe.transit_line_id == line])*fact_gpe

In [14]:
figs = {}
for mode in pt_modes:
    if mode != "bus":
        df = pd.DataFrame({
            "Ligne" : list(count_lines_gpe[mode].keys()),
            "Sans GPE" : list(list(count_lines_baseline[mode].values())),
            "Avec GPE" : list(list(count_lines_gpe[mode].values())),
        })

        df = df.sort_values(by='Avec GPE', ascending=True)

        figs[mode]= (px.bar(df.rename(columns={"Sans GPE" : "Référence"}), x='Ligne', y=['Référence', 'Avec GPE'],
                     #title=f'Nombre de trajets effectués par ligne. Mode : {mode}',
                     labels={'value': 'Nombre de trajets', 'variable': 'Simulation'},
                     barmode='group'))

In [15]:
figs_diff = {}
for mode in pt_modes:
    if mode != "bus":
        df = pd.DataFrame({
            "Ligne": list(count_lines_gpe[mode].keys()),
            "Sans GPE": list(count_lines_baseline[mode].values()),
            "Avec GPE": list(count_lines_gpe[mode].values()),
        })

        df['Différence'] = df['Avec GPE'] - df['Sans GPE']
        df = df[~df.Ligne.isin(["15", "16", "17", "18"])]

        df = df.sort_values(by='Différence', ascending=True)
        figs_diff[mode] = px.bar(df, x='Ligne', y='Différence',
                     title=f'Différence du nombre de trajets effectués par ligne. Mode : {mode}',
                     labels={'Différence': 'Différence du nombre de trajets'})


In [16]:
figs_percent = {}
for mode in pt_modes:
    if mode != "bus":
        df = pd.DataFrame({
            "Ligne": list(count_lines_gpe[mode].keys()),
            "Sans GPE": list(count_lines_baseline[mode].values()),
            "Avec GPE": list(count_lines_gpe[mode].values()),
        })

        df['Différence (%)'] = ((df['Avec GPE'] - df['Sans GPE']) / df['Sans GPE']) * 100

        df = df[~df.Ligne.isin(["15", "16", "17", "18"])]

        df = df.sort_values(by='Différence (%)', ascending=True)

        figs_percent[mode] = px.bar(df, x='Ligne', y='Différence (%)',
                     title=f'Différence en % du nombre de trajets effectués par ligne. Mode : {mode}',
                     labels={'Différence (%)': 'Différence en % du nombre de trajets'})

In [17]:
fig = figs["subway"]
fig.write_image(f"outputs/plots/comparaison/nb_trajets_line_subway.png", width=1000)
fig.show()

In [18]:
figs_diff["subway"].show()

In [19]:
mode = "subway"
fig = figs_percent[mode]
fig.write_image(f"outputs/plots/comparaison/nb_trajets_line_diff_pourcent_{mode}.png", width=1000)
fig.show()

In [20]:
fig = figs["rail"]
fig.write_image(f"outputs/plots/comparaison/nb_trajets_line_rail.png", width=1000)
fig.show()

In [21]:
figs_diff["rail"].show()

In [22]:
mode = "rail"
fig = figs_percent[mode]
fig.write_image(f"outputs/plots/comparaison/nb_trajets_line_diff_pourcent_{mode}.png", width=1000)
fig.show()

In [23]:
fig = figs["tram"]
fig.write_image(f"outputs/plots/comparaison/nb_trajets_line_tram.png", width=1000)
fig.show()

In [24]:
figs_diff["tram"].show()

In [25]:
mode = "tram"
fig = figs_percent[mode]
fig.write_image(f"outputs/plots/comparaison/nb_trajets_line_diff_pourcent_{mode}.png", width=1000)
fig.show()

In [26]:
#!jupyter nbconvert --no-input --to pdf comparaison_gpe.ipynb